In [ ]:
%matplotlib inline

from PIL import Image
import skvideo.io
import numpy as np
import torch
from torch.autograd import Variable
from torchvision import transforms
from tqdm import tqdm_notebook

from transformer_net import TransformerNet
from utils import recover_image, tensor_normalizer

In [ ]:
MODEL = "models/model_mosaic.pth"
transformer = TransformerNet()
transformer.load_state_dict(torch.load(MODEL))

In [ ]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
    tensor_normalizer()
])

## Low Resolution GIF Animation

Convert gif file to video file: 
```
ffmpeg -f gif -i cat.gif cat.mp4
```

In [ ]:
skvideo.io.ffprobe("videos/cat.mp4")

In [ ]:
frames = []
frames_orig = []
videogen = skvideo.io.vreader("videos/cat.mp4")
for frame in videogen:
    frames_orig.append(Image.fromarray(frame))
    frames.append(recover_image(transformer(
        Variable(preprocess(frame).unsqueeze(0), volatile=True)).data.numpy())[0])

In [ ]:
Image.fromarray(frames[3])

In [ ]:
writer = skvideo.io.FFmpegWriter("cat.mp4")# tuple([len(frames)] + list(frames[0].shape)))
for frame in frames:
    writer.writeFrame(frame)
writer.close()

## Higher Resolution Videos

In [ ]:
skvideo.io.ffprobe("videos/nz.mp4")

Switch to GPU:

In [ ]:
transformer.cuda()
BATCH_SIZE = 2

In [ ]:
batch = []
videogen = skvideo.io.FFmpegReader("videos/nz.mp4", {"-ss": "00:00:25", "-t": "00:00:15"})
writer = skvideo.io.FFmpegWriter("nz.mp4")
try:
    for frame in tqdm_notebook(videogen.nextFrame()):
        batch.append(preprocess(frame).unsqueeze(0))
        if len(batch) == BATCH_SIZE:
            for frame_out in recover_image(transformer(
                Variable(torch.cat(batch, 0), volatile=True).cuda()).data.cpu().numpy()):
                writer.writeFrame(frame_out)
            batch = []
except RuntimeError as e:
    pass
writer.close()

In [ ]:
batch = []
videogen = skvideo.io.FFmpegReader("videos/keelung_360.mp4", {"-ss": "00:00:25", "-t": "00:00:30"})
writer = skvideo.io.FFmpegWriter("keelung.mp4")
try:
    for frame in tqdm_notebook(videogen.nextFrame()):
        batch.append(preprocess(frame).unsqueeze(0))
        if len(batch) == BATCH_SIZE:
            for frame_out in recover_image(transformer(
                Variable(torch.cat(batch, 0), volatile=True).cuda()).data.cpu().numpy()):
                writer.writeFrame(frame_out)
            batch = []
except RuntimeError as e:
    pass
writer.close()